In [1]:
# Importing libraries
import pandas as pd
import re

import nltk
# TODO - @Harshit, check if these downloads can be removed (as they are now in local)
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
import yake
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import numpy as np
import torch
from numpy.linalg import norm
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from sentence_transformers import SentenceTransformer
from happytransformer import HappyTextToText, TTSettings

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hmishra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hmishra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hmishra\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Importing dataset to the workplace
dataset = pd.read_excel("FinalDatasetQA.xlsx")

## Data Cleaning
Here we define our methods to clean the user answer and model answer

In [3]:
# Cleaning Contractions
R_patterns = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'can not'),
    (r'(\w+)\'m', '\g<1> am'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)\'d like to', '\g<1> would like to'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are')
]

class REReplacer(object):
    def __init__(self, patterns = R_patterns):
        self.patterns = [(re.compile(regex), replace) for (regex, replace) in R_patterns]

    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        
        return s

In [4]:
# Lowering Text
def text_lower(text):
    return text.lower()

In [5]:
# Sentence Correction
def corrector(text):
    return text.replace('. ', '.')

In [6]:
# Cleaning Punctuations
def replace_punct(text):
    pattern = "[^\w\d\+\*\-\\\=\s]"
    replace = " "

    return re.sub(pattern, replace, text)

In [7]:
# Removing additional white spaces
def remove_extra(text):
    return " ".join(text.strip().split())

In [8]:
# Pre-processing the user and model answers

# TODO: @Raghav -> Writing it professionaly
rep_word = REReplacer() # object to remove contractions

dataset['Model Answer'] = dataset['Model Answer'].apply(rep_word.replace)
dataset['User Answer'] = dataset['User Answer'].apply(rep_word.replace)

# The below preprocessing is only for ner and keyword scoring units, hence withdrawing the inputs for similarity scoring unit
mod_ans = dataset.iloc[:, 2].values # List of Model Answers
usr_ans = dataset.iloc[:, 3].values # List of User Answers

dataset['Model Answer'] = dataset['Model Answer'].apply(text_lower)
dataset['User Answer'] = dataset['User Answer'].apply(text_lower)

dataset['Model Answer'] = dataset['Model Answer'].apply(corrector)
dataset['User Answer'] = dataset['User Answer'].apply(corrector)

dataset['Model Answer'] = dataset['Model Answer'].apply(replace_punct)
dataset['User Answer'] = dataset['User Answer'].apply(replace_punct)

dataset['Model Answer'] = dataset['Model Answer'].apply(remove_extra)
dataset['User Answer'] = dataset['User Answer'].apply(remove_extra)

## KeyWord Scoring Module
Adding the YAKE model to include keyword scoring

In [9]:
def yake_keywordExtractor(text):
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    deduplication_algo = "seqm"
    windowSize = 2
    numOfKeywords = 101

    custom_kw_extractor = yake.KeywordExtractor(lan = language, 
                                            n = max_ngram_size, 
                                            dedupLim = deduplication_threshold, 
                                            dedupFunc = deduplication_algo, 
                                            windowsSize = windowSize, 
                                            top = numOfKeywords, 
                                            features = None)

    return custom_kw_extractor.extract_keywords(text)
                                        

Adding Keyword Scoring Unit

In [10]:
def keyword_scoring(keywords_text1, keywords_text2):
    match = 0
    total = 0
    synonym_dict = [] # List to include synonym matching

    # TODO - @Raghav, check the correctness of comments in the following cell and add relevant comments

    for token in keywords_text1:
        total += token[1] # Total number of keywords in model answer

    if total == 0:
        return 10 # No keywords in model answer

    # Generating Synonyms for second text
    for var in keywords_text2:
        syn = wordnet.synsets(var[0]) # Synonym Dictionary
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(var[0])
        syn_words = list(set(syn_words))

        temp = []
        wt = word_tokenize(var[0])
        pos = pos_tag(wt)[0][1]

        for i in range(0, len(syn_words)):
            checker_wt = word_tokenize(syn_words[i])
            checker_pos = pos_tag(wt)[0][1]

            if pos == checker_pos:
                temp.append(syn_words[i])

        synonym_dict = synonym_dict + temp

    
    # Calculating the total number of matching keywords
    for token in keywords_text1:
        syn = wordnet.synsets(token[0])
        syn_words = [x.lemma_names() for x in syn]
        syn_words = [x for elem in syn_words for x in elem]
        syn_words.append(token[0])
        syn_words = list(set(syn_words))

        if len(set(syn_words).intersection(set(synonym_dict))) != 0:
            match += token[1]

    # Keyword score is number of matching keywords over total number of model keywords (normalized to 10)
    return match * 10 / total


## Similarity Scoring Module
Adding Similarity Scoring Unit

In [11]:
# Define Cosine Similarity
def cos_sim(sent1_emb, sent2_emb):
    cos = np.dot(sent1_emb, sent2_emb)/(norm(sent1_emb)*norm(sent2_emb))
    return cos

# Similarity Scoring
def similarity_scoring(text1, text2, model):
    mod_sent = sent_tokenize(text1) # Individual sentences in model answer
    usr_sent = sent_tokenize(text2) # Individual sentences in user answer

    mod_emb = [] # Incorporating Model sentences embeddings
    usr_emb = [] # Incorporating User sentences embeddings

    for sent in mod_sent:
        sent_emb = model.encode(sent)
        mod_emb.append(sent_emb)

    for sent in usr_sent:
        sent_emb = model.encode(sent)
        usr_emb.append(sent_emb)

    n = len(mod_sent)
    m = len(usr_sent)

    match = 0

    for i in range (0, n):
        for j in range (0, m):
            if cos_sim(mod_emb[i], usr_emb[j]) >= 0.75:
                # Defining cosine threshold at 0.75
                match += 1
                break

    return match / n

Adding Summarizer

In [12]:
# creating a t5 summarizer model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu') # Can replace to gpu for faster processing

def summarizer(text):
    preprocessed_text = text.strip().replace('\n', '')
    t5_input_text = 'summarize: ' + preprocessed_text
    
    tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)
    
    summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Adding Grammar Corrector

In [13]:
def grammar_corrector(text):
    happy_tt = HappyTextToText("T5", "prithivida/grammar_error_correcter_v1") # Loading the corrector library

    text = "gec: " + text # Necessary pre-processing

    settings = TTSettings(do_sample=True, top_k=10, temperature=0.5, min_length=1, max_length=100)
    result = happy_tt.generate_text(text, args=settings)
    
    return result.text

In [14]:
simcse = SentenceTransformer('princeton-nlp/sup-simcse-roberta-large') # SimCSE Model
sbert = SentenceTransformer('stsb-mpnet-base-v2')
hf = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

num_of_inputs = len(mod_ans) # Total number of Answers
sim_score = [0] * num_of_inputs # List to keep respective similarity score
sbert_score = [0] * num_of_inputs
hf_score = [0] * num_of_inputs

# TODO - @Raghav, try implementing multi-threading here
# Also check the viability, multi-threading not necessarily imply faster execution
for i in range(0, num_of_inputs):
    curr_mod = mod_ans[i]
    curr_usr = usr_ans[i]

    curr_usr = grammar_corrector(curr_usr) # grammar correction on user answer
    curr_usr += summarizer(curr_usr) # adding summary of user answer to user answer
    
    sim_val = similarity_scoring(curr_mod, curr_usr, simcse)
    sim_score[i] = sim_val * 10

    sbert_val = similarity_scoring(curr_mod, curr_usr, sbert)
    sbert_score[i] = sbert_val * 10

    hf_val = similarity_scoring(curr_mod, curr_usr, hf)
    hf_score[i] = hf_val * 10

# Converting list to numpy array
sim_score = np.array(sim_score)
sbert_score = np.array(sbert_score)
hf_score = np.array(hf_score)

No sentence-transformers model found with name C:\Users\hmishra/.cache\torch\sentence_transformers\princeton-nlp_sup-simcse-roberta-large. Creating a new one with MEAN pooling.
10/18/2023 14:42:14 - INFO - happytransformer.happy_transformer -   Using device: cpu
10/18/2023 14:42:15 - INFO - happytransformer.happy_transformer -   Moving model to cpu
10/18/2023 14:42:15 - INFO - happytransformer.happy_transformer -   Initializing a pipeline
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Named-Entity Recognition Module
Adding entities extraction unit

In [15]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

def ner_scoring(text1, text2):
    nlp = pipeline('ner', 
                   model=model, 
                   tokenizer=tokenizer, 
                   aggregation_strategy="simple") # Initializing the NLP pipeline

    mod = nlp(text1) # pre-processing model answer
    usr = nlp(text2) # pre-processing user answer

    mod_arr = []
    usr_arr = []

    for i in range(0, len(mod)):
        if(mod[i]['score'] >= 0.9):
            mod_arr.append(mod[i]['word'].lower())

    if len(mod_arr) == 0:
        return 10 # return full marks if no entities in model answer

    for i in range(0, len(usr)):
        usr_arr.append(usr[i]['word'].lower())

    mod_arr = set(mod_arr)
    usr_arr = set(usr_arr)

    return len(mod_arr.intersection(usr_arr)) * 10 / len(mod_arr)

In [16]:
# TODO - @Raghav, add relevant comment for the addition of these two attributes in dataset
# TODO - @Raghav, try adding multi-threading here as well
dataset['yake_keys_sentence1'] = dataset['Model Answer'].apply(yake_keywordExtractor)
dataset['yake_keys_sentence2'] = dataset['User Answer'].apply(yake_keywordExtractor)

# Adding the Keyword Score in dataset
dataset['keyword_score'] = dataset[['yake_keys_sentence1', 'yake_keys_sentence2']].apply(lambda x : keyword_scoring(x.yake_keys_sentence1,
                                                                                        x.yake_keys_sentence2),
                                                                                        axis = 1)
# TODO - @Raghav add similar comment on dropping these two additional attributes
dataset.drop(['yake_keys_sentence1', 'yake_keys_sentence2'], axis = 1)


# Adding the Similarity Score in dataset
dataset['similarity_score'] = sim_score.tolist()
dataset['hf_score'] = hf_score.tolist()
dataset['sbert_score'] = sbert_score.tolist()

# Generating and adding the NER Score in dataset
ner_score = [0] * num_of_inputs
for i in range(dataset.shape[0]):
    mod = dataset['Model Answer'][i] # Individual model answers
    usr = dataset['User Answer'][i] # Individual user answers

    ner_score[i] = ner_scoring(mod, usr)

ner_score = np.array(ner_score) # List to numpy array conversion
dataset['ner_score'] = ner_score.tolist()

In [19]:
# Saving the dataset in form of a csv file to be in neural network
dataset.to_csv('Scores.csv')